In [1]:
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
input = []
test = []
skip = True
with open('mfcc_test.csv', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in csvreader:
        if(skip):
            skip = False
            continue
        temp = []
        for col in row:
            temp = col.split(',')
        if(random.randint(1,10)%7!=0):
            input.append(temp)
        else:
            test.append(temp)
random.shuffle(input)
random.shuffle(test)
Y = [i[0] for i in input]
X = [(i[1:13]) for i in input]
for x in X:
    for i in range(0,12):
        x[i] = float(x[i])
test_Y = [i[0] for i in test]
test_X = [(i[1:13]) for i in test]
for t_x in test_X:
    for i in range(0,12):
        t_x[i] = float(t_x[i])
Y_train = []
for y in Y:
    output = 0
    number = 0
    for character in y:
        number = ord(character) - 65
        output += number
    output -= 2 * number
    Y_train.append(output)
Y_test = []
for y in test_Y:
    output = 0
    number = 0
    for character in y:
        number = ord(character) - 65
        output += number
    output -= 2 * number
    Y_test.append(output)
size = len(X)*(len(X[0]))

In [14]:
X_train = np.array(X)
Y_train = np.array(Y_train)
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train))
#train_dataset = torch.tensor(train_dataset, dtype = torch.float32)
X_test = np.array(test_X)
Y_test = np.array(Y_test)
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(Y_test))
#test_dataset = torch.tensor(test_dataset, dtype = torch.float32)
print(size)

948


In [4]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [6]:
input_dim = size
hidden_dim = 100
output_dim = 10
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [23]:
print(model.parameters())
print(len(list(model.parameters())))
print(list(model.parameters())[0].size())
print(list(model.parameters())[1].size())
print(list(model.parameters())[2].size())
print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x7fbe450a6e40>
4
torch.Size([100, 948])
torch.Size([100])
torch.Size([10, 100])
torch.Size([10])


In [24]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, size).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images.float())

        # Calculate Loss: softmax --> cross entropy loss
        outputs = outputs.view(list(model.parameters())[3].size())
        print(outputs.size(),labels.size(),images.size())
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities
                images = images.view(-1, size).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

torch.Size([10]) torch.Size([79]) torch.Size([1, 948])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)